In [1]:
# Imports 
import os
import sys
sys.path.append("..")
os.environ["pyna_language"] = 'julia'
import pdb
import numpy as np
import openmdao.api as om
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
%matplotlib inline
from pyNA.pyna import pyna

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Inputs 
z_cb_lst = np.hstack((np.linspace(25, 250, 10), np.linspace(300, 750, 10)))

TS_cb_lst = dict()
TS_cb_lst['stca']= dict()
TS_cb_lst['stca']['V2'] = np.linspace(0.75, 1.0, 11)
TS_cb_lst['stca']['Vmax'] = np.linspace(0.4, 1.0, 25)
TS_cb_lst['a10'] = dict()
TS_cb_lst['a10']['V2'] = np.linspace(0.775, 1.0, 10)
TS_cb_lst['a10']['Vmax'] = np.linspace(0.675, 1.0, 14)

Vmax_lst = dict()
Vmax_lst['stca'] = dict()
Vmax_lst['stca']['V2'] = 197*0.51444444
Vmax_lst['stca']['Vmax'] = 250*0.51444444
Vmax_lst['a10'] = dict()
Vmax_lst['a10']['V2'] = 234.4*0.51444444
Vmax_lst['a10']['Vmax'] = 250*0.51444444

### Compute trajectories

In [ ]:
# Compute trajectories for (z-TS) sweep 
if False:
        
    for ac_name in ['stca', 'a10']:

        for case in ['V2', 'Vmax']:
            
            py = pyna()
            
            py.case_name = ac_name
            py.ac_name = ac_name
            py.engine_deck_name = 'engine_deck_' + ac_name + '.csv'
            py.save_results = True
            py.output_directory_name = 'STCBx/' + case
            py.tolerance = 1e-6
            py.max_iter = 300
            py.TS_to = 1.0
            py.TS_vnrs = 1.0
            py.pkrot = False

            py.theta_flaps = 10.
            py.theta_slats = -6.
            
            for j,TS in enumerate(TS_cb_lst[ac_name][case]):
                for i, z_cb in enumerate(z_cb_lst):

                    print("TS: ", TS)
                    print("z_cb:", z_cb)

                    py.TS_cb = TS
                    py.z_cb = z_cb
                    py.output_file_name = py.ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.sql'
                    
                    # Compute trajectory
                    py.v_max = Vmax_lst[ac_name][case]
                                        
                    converged = py.compute_trajectory(trajectory_mode='cutback', objective='x_end')

                    # Save timeseries
                    path_save_name = 'trajectory_' + py.ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.csv'
                    engine_save_name = 'engine_' + py.ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.csv'
                    py.save_time_series(py.path, py.airframe, path_save_name, engine_save_name)
                    
                    clear_output()

TS:  0.75
z_cb: 75.0
--- Linkage Report [phases] ---
    --- groundroll - rotation ---
        time           [final] ==  time           [initial]
        states:x       [final] ==  states:x       [initial]
        states:v       [final] ==  states:v       [initial]
        states:alpha   [final] ==  states:alpha   [initial]
    --- rotation - liftoff ---
        time               [final] ==  time             [initial]
        states:x           [final] ==  states:x         [initial]
        parameters:z       [final*] ==  states:z         [initial]
        states:v           [final] ==  states:v         [initial]
        states:alpha       [final] ==  controls:alpha   [initial]
        parameters:gamma   [final*] ==  states:gamma     [initial]
    --- liftoff - vnrs ---
        time             [final] ==  time             [initial]
        states:x         [final] ==  states:x         [initial]
        states:v         [final] ==  states:v         [initial]
        controls:alpha   

In [4]:
# Plot trajectories 
if False: 
    ac_name = 'stca'
    case = 'Vmax'
    
    py.ac_name = ac_name

    fig, ax = plt.subplots(2, 2, figsize=(20,8))
    plt.style.use('../utils/plot.mplstyle')

    for j,TS in enumerate(TS_cb_lst[ac_name][case]):
        for i, z_cb in enumerate(z_cb_lst):

            py.output_file_name = py.ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.sql'

            py.engine_file_name = 'engine_' + py.ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.csv'
            py.trajectory_file_name = 'trajectory_' + py.ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.csv'

            data = pd.read_csv('../cases/'+ac_name+'/trajectory/STCB/'+case+'/'+py.trajectory_file_name)

            ax[0, 0].plot(data['X [m]'], data['Z [m]'])
            ax[1, 0].plot(data['X [m]'], data['TS [-]'])
            ax[0, 0].grid(True)
            ax[1, 0].grid(True)

            ax[0, 1].plot(data['X [m]'], data['gamma [deg]'])
            ax[1, 1].plot(data['X [m]'], data['V [m/s]'])
            ax[0, 1].grid(True)
            ax[1, 1].grid(True)        

### Extract properties of the trajectories

In [5]:
# Compute noise time series
if False:
    
    k_rot_lst = dict()
    LD_climbout = dict()
    
    for ac_name in ['a10']:

        k_rot_lst[ac_name] = dict()
        LD_climbout[ac_name] = dict()
        
        for case in ['Vmax']:
            
            k_rot_lst[ac_name][case] = np.zeros((np.size(z_cb_lst), np.size(TS_cb_lst[ac_name][case])))
            LD_climbout[ac_name][case] = np.zeros((np.size(z_cb_lst), np.size(TS_cb_lst[ac_name][case])))
            
            py = pyna()
            
            py.output_directory_name = 'STCB/' + case
            py.case_name = ac_name
            py.ac_name = ac_name
            py.engine_deck_name = 'engine_deck_' + ac_name + '.csv'
            py.save_results = True
            py.output_directory_name = 'STCB/' + case
            
            for j,TS in enumerate(TS_cb_lst[ac_name][case]):
                for i, z_cb in enumerate(z_cb_lst):

                    py.output_file_name = py.ac_name + '_' + str(np.round(z_cb, 3)) + '_' + str(np.round(TS, 3)) + '.sql'
                    
                    print(ac_name, " ", case, " - TS: ", TS, "; z_cb: ", z_cb)

                    results = py.load_results(py.output_file_name)
                    
                    k_rot_lst[ac_name][case][i,j] = results.get_val('phases.groundroll.parameters:k_rot')
                    LD_climbout[ac_name][case][i,j] = np.max(results.get_val('phases.cutback.interpolated.c_l')/results.get_val('phases.cutback.interpolated.c_d'))

            np.save('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/k_rot.npy', k_rot_lst[ac_name][case])
            np.save('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/LD_climbout.npy', LD_climbout[ac_name][case])

            clear_output()

### Compute certification noise levels

In [ ]:
# Compute noise time series
if True:
        
    for ac_name in ['a10']:

        for case in ['V2', 'Vmax']:
            
            n_sl = 131
            epnl_sideline = np.zeros((np.size(z_cb_lst), np.size(TS_cb_lst[ac_name][case]), n_sl))
            epnl_lateral = np.zeros((np.size(z_cb_lst), np.size(TS_cb_lst[ac_name][case])))
            epnl_flyover = np.zeros((np.size(z_cb_lst), np.size(TS_cb_lst[ac_name][case])))
            
            py = pyna()
            py.ac_name = ac_name
            py.case_name = ac_name
            py.output_directory_name = 'STCB/' + case
            
            for j,TS in enumerate(TS_cb_lst[ac_name][case]):
                for i, z_cb in enumerate(z_cb_lst):

                    print(ac_name, " ", case, " - TS: ", TS, "; z_cb: ", z_cb)

                    py.TS_cb = TS
                    py.z_cb = z_cb
                    py.output_file_name = py.ac_name + '_' + str(np.round(z_cb, 3)) + '_' + str(np.round(TS, 3)) + '.sql'
                    py.engine_timeseries_name = 'engine_' + py.ac_name + '_' + str(np.round(z_cb, 3)) + '_' + str(np.round(TS, 3)) + '.csv'
                    py.trajectory_file_name = 'trajectory_' + py.ac_name + '_' + str(np.round(z_cb, 3)) + '_' + str(np.round(TS, 3)) + '.csv'
                    
                    py.observer_lst = ['contours',]

                    py.lateral_attenuation = True
                    py.shielding = False
                    py.all_sources = False
                    py.fan_inlet_source = True
                    py.fan_discharge_source = True
                    py.core_source = True
                    py.jet_mixing_source = True
                    py.jet_shock_source = False
                    py.airframe_source = True
                    
                    x_observer_lst = np.zeros((n_sl+1, 3))
                    x_observer_lst[:-1, 0] = np.linspace(0, 6500, n_sl)
                    x_observer_lst[:-1, 1] = 450*np.ones(n_sl)
                    x_observer_lst[:, 2] = 4*0.3048*np.ones(n_sl+1)
                    x_observer_lst[-1,0] = 6500.

                    py.x_observer_array = x_observer_lst

                    py.compute_noise_timeseries()

                    epnl_sideline[i,j,:] = py.noise_timeseries.get_val('noise.epnl')[:-1]
                    epnl_lateral[i,j] = np.max(py.noise_timeseries.get_val('noise.epnl')[:-1])
                    epnl_flyover[i,j] = py.noise_timeseries.get_val('noise.epnl')[-1]

            np.save('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps_lat/epnl_lateral.npy', epnl_lateral)
            np.save('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps_lat/epnl_flyover.npy', epnl_flyover)
            np.save('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps_lat/epnl_sideline.npy', epnl_sideline)

            clear_output()

a10   Vmax  - TS:  0.675 ; z_cb:  25.0
Model viewer data has already has already been recorded for Driver.
a10   Vmax  - TS:  0.675 ; z_cb:  50.0
Model viewer data has already has already been recorded for Driver.
a10   Vmax  - TS:  0.675 ; z_cb:  75.0
Model viewer data has already has already been recorded for Driver.
a10   Vmax  - TS:  0.675 ; z_cb:  100.0
Model viewer data has already has already been recorded for Driver.
a10   Vmax  - TS:  0.675 ; z_cb:  125.0
Model viewer data has already has already been recorded for Driver.
